In [ ]:
import librosa
import librosa.display
import IPython.display as ipd0
from IPython.display import Audio
import os
import scipy.io.wavfile as waves #biblioteca para procesar audio .wav en python
import matplotlib.pyplot as plt #biblioteca para graficar
import scipy as sp # for signal processing
from scipy import signal
import IPython.display as ipd
import numpy as np
import random
import makelab
from makelab import signal
from makelab import audio
%matplotlib inline

Libreria makelab diseñada y escrita por el profesor Jon E. Froehlich de la Univerdidad de Washington con ayuda de estudiantes con permiso de uso libre para fines educativos

In [ ]:
# INGRESO
# archivo = input('archivo de audio: ')
'''
arch=input('ingrese el nombre del archivo: ')
archivo=(arch +'.wav')
'''
archivo = 'Alarm01.wav' #archivo de entrada

In [ ]:
# PROCEDIMIENTO
fs, sonido = sp.io.wavfile.read(archivo) #biblioteca lee el archivo de audio y devuelve la frecuencia de muestreo y la cuantizacion para la cantidad de muestras del (monofonoico y estrereo)
tamano_sonido = len(sonido)#devuelve la cantidad de simbolos v(k) del audio digitalizado
#trabajar con un solo canal de audio
tamano = np.shape(sonido) #divide ambos vectores de canales
#muestras = tamano[0]
m = len(tamano) #cantidad de canales de las muestras de audio
canales = 1  # monofónico
if (m>1):  # estéreo
    canales = tamano[1]
# experimento con un canal
if (canales>1):
    canal = 0
    vT = sonido[:,canal] 
else:
    vT = sonido
#vT corresponde a la senal cuantizada para el canal de audio seleccionado
print(vT)
longitud_del_audio_en_sec=vT.shape[0]/fs
bits = 16 #cantidad de bits para cuantizacion


In [ ]:
print(f"Frecuencia de muestreo: {fs} Hz")
print(f"Numero de canales de senal = {len(sonido.shape)}")
print(f"Cantidad de muestras del canal: {sonido.shape[0]}")
print(f"Duracion del audio = {longitud_del_audio_en_sec}s")
print(f"{bits}-Los niveles de cuantizacion van de -{2**(bits - 1)} a {2**(bits - 1) - 1}")
print(f"Valor de amplitutud maximo de la senal de audio: {np.max(vT)}")

## Senal con nivel de cuantizacion original

In [ ]:
xlim_zoom = (40000, 80000) # zoom de la senal muestreada
makelab.signal.plot_signal(vT, fs, bits, xlim_zoom = xlim_zoom) #plotear vector de bits que muestran los valores cuantizados vs la cantidad de muestras y vs el tiempo
ipd.Audio(vT, rate=fs)

In [ ]:
# Codificador
# Los archivos de audio .wav codificados en PCM utilizan una cuantizacion
# en 16 bits con rango = 1, es decir, usan 65536 valores de amplitud, desde 
# -32768 hasta 32767. Entonces, para ser fiel al formato, se adoptan estos
# parametros.

vT = senal_real # Señal muestreada

# Se le asigna un número decimal a cada una de las muestras
# Se considera: -32768 = 0 ... 32767 = 65536
bkT = np.zeros(len(vT))

# Conversión de muestras a símbolos
for i in range(0,len(bkT)):
    bkT[i] = (2**16)/2 + vT[i]
    
# Se aplica una codificacion de longitud fija convirtiendo
# el numero decimal de cada simbolo a binario de 16 bits

bfT = np.zeros(16*len(bkT)) # vector para los bits

# Conversión de decimal a binario de los simbolos
for i in range(0,len(bkT)):
    num = bkT[i]
    for j in range(0,16):
        if num >= 2**(16-1-j):
            bfT[16*i+j] = 1
            num = num - 2**(16-1-j)
        else:
            bfT[16*i+j] = 0

bfT = bfT.astype(int) # bits a la salida del codificador

# Asumiendo un canal ideal:
bfR = bfT

#-----------------------------------------------------

# Decodificador

# Para convertir bits a símbolos:
cant_simb = 2**16 # Cantidad de símbolos
bkR = np.zeros(len(t)) # Arreglo para los símbolos

# Pasar de binario a decimal
for i in range(0,len(bkR)):
    for j in range(0,16):
        bkR[i] += bfR[16*i+j]*(2**(16-j-1)) 
    
bkR = bkR.astype(int) # Símbolos

# Convertir de símbolos a muestras
vR = np.zeros(len(bkR)) # vector para las muestras recuperadas

for i in range(0,len(vR)):
    vR[i] = -(2**16)/2 + bkR[i]
    
vR = vR.astype(int)

# Despliegue de resultados

t_bits = np.arange(0,len(bfT))
plt.figure(1)
plt.suptitle('Bits a la salida del codificador')
plt.plot(t_bits, bfT, drawstyle='steps-pre')
plt.xlabel('número de bit')
plt.ylabel('bits')
plt.xlim(1500,1650)
plt.show()

In [ ]:
makelab.signal.plot_signal(vR, fs, bits, xlim_zoom = xlim_zoom)
ipd.Audio(vR, rate=fs)

In [ ]:
#escribir y guardar archivo .wav
print('archivo de parte[] grabado...')
waves.write('resultado.wav', fs, vR.astype(np.int16))